#### For each XAI method and Run copy and paste the top k features into the Trial list (also possible to add more Trials if you would like)

In [2]:

###########################################################################################################################################################################################################################################################################################################################
print('---------------------------------------------------------------------------------')
print('DNN DeepLIFT NSL GLOBAL')
print('---------------------------------------------------------------------------------')
Trial_1 = [
'service_http',
'dst_host_srv_serror_rate',
'dst_host_serror_rate',
'flag_S0',
'serror_rate',
'srv_serror_rate',
'flag_SF',
'same_srv_rate',
'rerror_rate',
'count',
'dst_host_srv_count',
'logged_in',
'dst_host_same_src_port_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'hot',
'srv_rerror_rate',
'service_private',
'dst_host_count',
'dst_host_same_srv_rate'
]

Trial_2 = [
'logged_in',
'dst_host_rerror_rate',
'Protocol_type_tcp',
'dst_host_srv_rerror_rate',
'srv_rerror_rate',
'dst_host_same_srv_rate',
'dst_host_srv_serror_rate',
'dst_host_serror_rate',
'dst_host_srv_count',
'dst_host_count',
'rerror_rate',
'srv_serror_rate',
'Protocol_type_udp',
'flag_S0',
'flag_SF',
'same_srv_rate',
'service_http',
'flag_REJ',
'count',
'dst_host_same_src_port_rate'
]

Trial_3 = [
'logged_in',
'dst_host_count',
'count',
'service_private',
'service_http',
'srv_diff_host_rate',
'flag_S0',
'srv_serror_rate',
'diff_srv_rate',
'flag_SF',
'rerror_rate',
'flag_REJ',
'same_srv_rate',
'dst_host_rerror_rate',
'dst_host_same_srv_rate',
'srv_count',
'dst_host_srv_rerror_rate',
'service_domain_u',
'dst_host_srv_serror_rate',
'Protocol_type_tcp'
]

Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

print('---------------------------------------------------------------------------------')
print('DNN IG NSL GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'service_http',
 'srv_serror_rate',
 'serror_rate',
 'flag_SF',
 'flag_S0',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'same_srv_rate',
 'dst_host_count',
 'count',
 'logged_in',
 'Protocol_type_icmp',
 'service_private',
 'service_ecr_i',
 'dst_host_rerror_rate',
 'service_smtp',
 'dst_host_srv_rerror_rate',
 'dst_host_diff_srv_rate'
]

Trial_2 = [
'logged_in',
 'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'Protocol_type_udp',
 'srv_serror_rate',
 'flag_S0',
 'dst_host_same_srv_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'service_private',
 'flag_SF',
 'serror_rate',
 'dst_host_diff_srv_rate',
 'Protocol_type_tcp',
 'same_srv_rate',
 'dst_host_rerror_rate',
 'count',
 'dst_host_srv_rerror_rate',
 'dst_host_same_src_port_rate',
 'service_http'
]
Trial_3 =[
'dst_host_count',
 'logged_in',
 'srv_serror_rate',
 'flag_S0',
 'count',
 'service_private',
 'srv_diff_host_rate',
 'flag_SF',
 'same_srv_rate',
 'diff_srv_rate',
 'rerror_rate',
 'dst_host_srv_rerror_rate',
 'Protocol_type_tcp',
 'dst_host_same_srv_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_serror_rate',
 'srv_count',
 'service_http',
 'serror_rate',
 'flag_REJ'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)


print('---------------------------------------------------------------------------------')
print('DNN LRP NSL GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'service_http',
 'srv_serror_rate',
 'serror_rate',
 'flag_SF',
 'dst_host_srv_count',
 'flag_S0',
 'dst_host_same_src_port_rate',
 'same_srv_rate',
 'dst_host_count',
 'count',
 'logged_in',
 'Protocol_type_icmp',
 'service_private',
 'service_ecr_i',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'service_smtp',
 'dst_host_diff_srv_rate'
]

Trial_2 = [
'dst_host_count',
 'logged_in',
 'srv_serror_rate',
 'count',
 'flag_S0',
 'flag_SF',
 'same_srv_rate',
 'service_private',
 'srv_diff_host_rate',
 'Protocol_type_tcp',
 'dst_host_srv_rerror_rate',
 'rerror_rate',
 'dst_host_same_srv_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_serror_rate',
 'diff_srv_rate',
 'srv_count',
 'service_http',
 'serror_rate',
 'service_domain_u'
]
Trial_3 =[
'logged_in',
 'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'srv_serror_rate',
 'Protocol_type_udp',
 'dst_host_same_srv_rate',
 'dst_host_count',
 'flag_S0',
 'dst_host_srv_count',
 'service_private',
 'flag_SF',
 'serror_rate',
 'same_srv_rate',
 'count',
 'dst_host_diff_srv_rate',
 'service_http',
 'dst_host_rerror_rate',
 'dst_host_same_src_port_rate',
 'Protocol_type_tcp',
 'dst_host_srv_rerror_rate'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

---------------------------------------------------------------------------------
DNN DeepLIFT NSL GLOBAL
---------------------------------------------------------------------------------
0.65
---------------------------------------------------------------------------------
DNN IG NSL GLOBAL
---------------------------------------------------------------------------------
0.65
---------------------------------------------------------------------------------
DNN LRP NSL GLOBAL
---------------------------------------------------------------------------------
0.65
